In [ ]:
from src.util import DataLoader, AortaNormalizer, lowpass_filter
from src.visualiazation import (
    plot_pca,
    plot_random_predictions,
    plot_relative_error_aorta,
)
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

**Load Data**

In [ ]:
data_path = "/data/PulHypStudie_Check_npz_v2/"

In [ ]:
data_list = [i for i in range(1, 10)]
data_list  # pig 10 -> test pig

In [ ]:
data_loader = DataLoader(data_path)

X, Y, Pig = data_loader.load_data(data_list)

In [ ]:
aorta_normalizer = AortaNormalizer()
Y_norm = aorta_normalizer.normalize_forward(Y)
Y_true = Y[:, :, 0]

assert np.allclose(Y_true, aorta_normalizer.normalize_inverse(Y_norm)[:, :, 0])

**Load VAE**

In [ ]:
import keras_tuner as kt

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    BatchNormalization,
    Activation,
    Conv1D,
    ZeroPadding1D,
    Reshape,
    Cropping1D,
)
from tensorflow.keras.layers import Conv1DTranspose
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import Mean


class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, beta, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta
        self.total_loss_tracker = Mean(name="total_loss")
        self.reconstruction_loss_tracker = Mean(name="reconstruction_loss")
        self.kl_loss_tracker = Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            loss, reconstruction_loss, kl_loss = self.vae_loss(
                data, reconstruction, z_mean, z_log_var
            )

        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

        self.total_loss_tracker.update_state(loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        if isinstance(data, tuple):
            data = data[0]

        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        loss, reconstruction_loss, kl_loss = self.vae_loss(
            data, reconstruction, z_mean, z_log_var
        )

        self.total_loss_tracker.update_state(loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        return self.decoder(z)

    def vae_loss(self, inputs, outputs, z_mean, z_log_var):
        mse_loss_fn = MeanSquaredError()
        input_dim = 1024
        reconstruction_loss = mse_loss_fn(inputs, outputs) * input_dim
        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        )
        total_loss = reconstruction_loss + self.beta * kl_loss
        return total_loss, reconstruction_loss, kl_loss


# The encoder model
def encoder_model(
    input_shape=(1024, 1),
    channels=(5, 10, 20, 30),
    strides=(4, 4, 4, 4),
    kernel_size=(5, 5, 5, 5),
    latent_dim=8,
):
    encoder_inputs = Input(shape=input_shape)
    x = encoder_inputs

    for ch_n, str_n, kernel_s in zip(channels, strides, kernel_size):
        x = Conv1D(ch_n, kernel_s, padding="same", strides=1)(x)
        x = BatchNormalization()(x)
        x = Activation("elu")(x)

        x = Conv1D(ch_n, kernel_s, padding="same", strides=str_n)(x)
        x = BatchNormalization()(x)
        x = Activation("elu")(x)

    x = Flatten()(x)

    z_mean = Dense(latent_dim, name="z_mean")(x)
    z_log_var = Dense(latent_dim, name="z_log_var")(x)

    z = Sampling()((z_mean, z_log_var))

    return encoder_inputs, z_mean, z_log_var, z


# The decoder model
def decoder_model(
    latent_dim=8,
    channels=(30, 20, 10, 5),
    strides=(4, 4, 4, 4),
    kernel_size=(5, 5, 5, 5),
):
    latent_inputs = Input(shape=(latent_dim,))
    L = (1024 // np.prod(strides)) * channels[0]
    x = Dense(L, activation="elu")(latent_inputs)
    x = Reshape((1024 // np.prod(strides), channels[0]))(x)

    for ch_n, str_n, kernel_s in zip(channels, strides, kernel_size):
        x = Conv1DTranspose(ch_n, kernel_s, padding="same", strides=str_n)(x)
        x = BatchNormalization()(x)
        x = Activation("elu")(x)

        x = Conv1D(ch_n, kernel_s, padding="same", strides=1)(x)
        x = BatchNormalization()(x)
        x = Activation("elu")(x)

    x = Conv1DTranspose(1, 1, activation="elu", padding="same")(x)
    decoded = x

    return latent_inputs, decoded


def build_vae_model(hp):
    latent_dim = hp.Int("latent_dim", min_value=4, max_value=16, step=4)
    beta = hp.Float("beta", min_value=0.1, max_value=2.0, step=0.1)
    # num_channels_choice = hp.Choice("num_channels", values=[5, 10, 20, 30])
    kernel_size = hp.Int("kernel_size", min_value=3, max_value=9, step=1)
    strides = hp.Int("strides", min_value=2, max_value=4, step=1)

    channels = (5, 10, 20, 30)
    # kernel_sizes=(5, 5, 5, 5)
    kernel_sizes = [kernel_size] * 4
    # strides=(4, 4, 4, 4)
    stride_sizes = [strides] * 4

    # Build encoder and decoder
    encoder_inputs, z_mean, z_log_var, z = encoder_model(
        channels=channels,
        kernel_size=kernel_sizes,
        strides=stride_sizes,
        latent_dim=latent_dim,
    )
    encoder = Model(encoder_inputs, (z_mean, z_log_var, z), name="Encoder")

    decoder_inputs, decoder_outputs = decoder_model(
        channels=channels[::-1],
        kernel_size=kernel_sizes[::-1],
        strides=stride_sizes[::-1],
        latent_dim=latent_dim,
    )
    decoder = Model(decoder_inputs, decoder_outputs, name="Decoder")

    encoder.summary()
    decoder.summary()
    vae = VAE(encoder, decoder, beta=beta)

    vae.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("lr", [1e-3, 1e-4, 1e-5])
        )
    )
    return vae


# Instantiate the tuner
tuner = kt.Hyperband(
    build_vae_model,
    objective="loss",
    max_epochs=20,
    factor=2,
    directory="vae_hpt",
    project_name="vae_tuning_1",
)

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
vae = tuner.hypermodel.build(best_hps)

In [ ]:
# sel_model = "src/weights/vae_model_excl10_0.weights.h5"
# sel_model = "src/weights/vae_model_excl_none_1.weights.h5"
sel_model = "src/weights/vae_model_excl_10_2.weights.h5"

# vae = vae_model()
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.load_weights(sel_model)
_, _, z = vae.encoder.predict(Y_norm)

In [ ]:
plot_pca(z)

**Train Mapper**

- [KerasTuner](https://www.tensorflow.org/tutorials/keras/keras_tuner)

In [ ]:
import keras_tuner as kt
from tensorflow import keras

In [ ]:
def build_model(hp):
    latent_dim = 8
    # initialize the sequential model.
    model = keras.Sequential()
    # input layer
    model.add(keras.layers.Input(shape=(64, 1024, 1)))

    # tune the number of hidden layers and units in each.
    for i in range(1, hp.Int("num_layers", 4, 7)):
        print(f"Init layer {i=}")
        hp_units = hp.Int("units_" + str(i), min_value=2, max_value=16, step=4)
        hp_kernel = hp.Int("kernel_" + str(i), min_value=2, max_value=9, step=1)
        # stride dim (0,1)
        hp_strides_0 = hp.Int("units_0_" + str(i), min_value=1, max_value=4, step=1)
        hp_strides_1 = hp.Int("units_1_" + str(i), min_value=2, max_value=4, step=1)
        hp_activation = hp.Choice(
            "activation_" + str(i), values=["relu", "elu", "tanh"]
        )
        hp_dropout = hp.Float("dropout_" + str(i), 0, 1.0, step=0.1)

        # create layer
        model.add(
            keras.layers.Conv2D(
                hp_units,
                hp_kernel,
                strides=(hp_strides_0, hp_strides_1),
                padding="same",
            )
        )
        model.add(keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation(hp_activation))
        model.add(keras.layers.Dropout(hp_dropout))

    model.add(keras.layers.Flatten())
    # output layer.
    model.add(keras.layers.Dense(latent_dim, activation="linear"))

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 1e-5])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.MeanAbsoluteError(),
        # loss=keras.losses.MeanSquaredError(),
        metrics=["accuracy"],
    )
    print(model.summary())

    return model

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=50,
    factor=2,
    directory="hpt_mapper_test10",
    project_name="hpt_mapper_test10",
)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)

In [ ]:
tuner.search(
    X, z, epochs=50, batch_size=20, validation_split=0.2, callbacks=[stop_early]
)

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_hps.values

**Load Best Model**

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
X_train, X_valid, z_train, z_valid = train_test_split(
    X, z, test_size=0.2, random_state=42
)
print(X_train.shape, X_valid.shape, z_train.shape, z_valid.shape)

In [ ]:
# es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)
history = model.fit(
    X_train,
    z_train,
    epochs=100,
    batch_size=32,
    # callbacks=[es],
    validation_data=(X_valid, z_valid),
)

In [ ]:
model.save_weights("src/weights/mapper_model_1.weights.h5")
np.savez("src/weights/mapper_model_1_history.npz", history=history)

**Test model**

In [ ]:
model.evaluate(X, z)

In [ ]:
X_test, Y_t, _ = data_loader.load_data(10, shuffle=False)

aorta_normalizer = AortaNormalizer()
Y_norm = aorta_normalizer.normalize_forward(Y_t)
Y_true = Y_t[:, :, 0]

assert np.allclose(Y_true, aorta_normalizer.normalize_inverse(Y_norm)[:, :, 0])

In [ ]:
# predict with trained model
z_pred = model.predict(X_test)
Y_pred = vae.decoder.predict(z_pred)
Y_pred = aorta_normalizer.normalize_inverse(Y_pred)[:, :, 0]

In [ ]:
rel_err_config = {
    "std": True,
    "var": False,
    "mean": True,
    "s_name": "images/result_1.png",
}

plot_relative_error_aorta(Y_true, Y_pred, **rel_err_config)

In [ ]:
plot_random_predictions(Y_true, Y_pred, n=10)

In [ ]:
from src.LaTeX_export import output_err_for_LaTeX, output_curve_for_LaTeX

In [ ]:
output_err_for_LaTeX(Y_true, Y_pred, f_name="err_serious_result_1.csv")

In [ ]:
output_curve_for_LaTeX(Y_true, Y_pred, f_name="curve_serious_result_1.csv")

In [ ]:
Y_true.shape

In [ ]:
DAP = np.min(Y_true, axis=1) - np.min(Y_pred, axis=1)
SAP = np.max(Y_true, axis=1) - np.max(Y_pred, axis=1)
MAP = np.mean(Y_true, axis=1) - np.mean(Y_pred, axis=1)

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
dd = {"DAP": DAP, "SAP": SAP, "MAP": MAP}
DF = pd.DataFrame(dd)

In [ ]:
sns.histplot(DF)
plt.savefig("hist_DapSapMap.png")

In [ ]:
sns.boxplot(DF)
plt.grid()
plt.savefig("box_DapSapMap.png")

**Value over time**

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(lowpass_filter(np.concatenate(Y_true)[:10_000]), label="Pred")
plt.plot(lowpass_filter(np.concatenate(Y_pred)[:10_000]), label="True")
plt.legend()

In [ ]:
from plotLaTeX import LinePlot

In [ ]:
Lplt = LinePlot()

In [ ]:
Lplt.add_yvals(lowpass_filter(np.concatenate(Y_true)[: 1024 * 9]), y_name="True")
Lplt.add_yvals(lowpass_filter(np.concatenate(Y_pred)[: 1024 * 9]), y_name="Pred")

In [ ]:
Lplt.export(f_name="curve_results.csv")